## scikeras Keras regressor integration

The integration of KerasRegressor in pipeline is limited. To use full keras capabilities, you may need to modify the pipeline.

In [1]:
from pathlib import Path
import glob
import gzip
import os
import pandas as pd

for folder in ['regression']:#, 'classif']:
    rootdir = Path('rawdata/'+folder)
    file_list = [f for f in rootdir.glob('**/*') if f.is_dir()]
    files_pattern = ["*Xcal*","*Ycal*","*Xval*","*Yval*"]
    for dir in file_list:
        training_name = str(dir)
        projdir = Path(dir)
        for f in files_pattern:
            file = next(projdir.glob(f))
            new_file = str(file).replace("rawdata","data2")
            new_file += '.gz'
            if os.path.exists(new_file):
                print("skipping", file)
                continue
            df = pd.read_csv(file, sep=";", header=0, skip_blank_lines=False)
            new_folder = str(file.parent).replace("rawdata","data2")
            if not os.path.exists(new_folder):
                os.makedirs(new_folder)
            df.to_csv(new_file, sep=";", index=False, compression="gzip")
            print(file, "done")

skipping rawdata\regression\Wood_Sustainability_511_Davrieux_RMSE0.092\Xcal.csv
skipping rawdata\regression\Wood_Sustainability_511_Davrieux_RMSE0.092\Ycal.csv
skipping rawdata\regression\Wood_Sustainability_511_Davrieux_RMSE0.092\Xval.csv
skipping rawdata\regression\Wood_Sustainability_511_Davrieux_RMSE0.092\Yval.csv


In [7]:
from pathlib import Path
import glob
import gzip
import os
import pandas as pd

## un fichier par dataset > run_idx + datasheet + link vers runs & link vers tensorboard
## un fichier global json
## un export csv ligne=run colonnes=[datasetx(RMSE,R²)]

def parse(json_file):
    pass

# r=root, d=directories, f = files
for r, d, f in os.walk("results"):
    for folder in d:
        # folders.append(os.path.join(r, folder))
        file_list = [f for f in Path(os.path.join(r, folder)).glob('*.json') if not f.is_dir()]
        if len(file_list) > 0:
            print(folder, ">", file_list)
            for file in file_list:
                parse(file)



_inv_rmse > [WindowsPath('results/_inv_rmse/ALPINE_Calpine_424_Murguzur_results.json'), WindowsPath('results/_inv_rmse/ALPINE_C_424_Murguzur_results.json'), WindowsPath('results/_inv_rmse/ALPINE_Nalpine_552_Murguzur_results.json'), WindowsPath('results/_inv_rmse/ALPINE_N_552_Murguzur_results.json'), WindowsPath('results/_inv_rmse/ALPINE_Palpine_291_Murguzur_results.json'), WindowsPath('results/_inv_rmse/ALPINE_P_291_Murguzur_results.json'), WindowsPath('results/_inv_rmse/Cassava_TBC_3393_Sanchez_results.json'), WindowsPath('results/_inv_rmse/Cassava_TBC_3432_Shen_results.json'), WindowsPath('results/_inv_rmse/Cassava_TBC_3556_Davrieux_results.json'), WindowsPath('results/_inv_rmse/Cassava_TTC_3830_Davrieux_results.json'), WindowsPath('results/_inv_rmse/Eucalyptus_Density_1654_Chaix_results.json'), WindowsPath('results/_inv_rmse/LUCAS_SOCcropland_8731_Nocita_results.json'), WindowsPath('results/_inv_rmse/LUCAS_SOCgrassland_4096_Nocita_results.json'), WindowsPath('results/_inv_rmse/LUCAS